In [ ]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import datetime, re, time, itertools, smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import requests

In [ ]:
start_date = datetime.datetime(2021,12,24)
days_range = 2
trip_length, min_length, max_length = 7, 6, 12
price_goal = 5000
cities = 'YMQ-OSL'

In [ ]:
start = pd.date_range(start_date, periods=days_range).to_pydatetime().tolist()
end = pd.date_range(start_date + datetime.timedelta(days=trip_length), periods=days_range).to_pydatetime().tolist()

#Get all combinations of dates
all_dates = list(itertools.product(start,end))

In [ ]:
departing=[]
returning=[]
for i in range(len(all_dates)):
    if (([x[1] for x in all_dates][i] - [x[0] for x in all_dates][i]).days > min_length) & \
       (([x[1] for x in all_dates][i] - [x[0] for x in all_dates][i]).days < max_length):
            
        departing.append(([x[0] for x in all_dates][i]).strftime('%Y-%m-%d'))
        returning.append(([x[1] for x in all_dates][i]).strftime('%Y-%m-%d'))
    else:
        None

In [ ]:
#Cleaning dates to make them usable in loop
for i in range(len(departing)):
    departing[i]=re.sub(' 00:00:00', '', departing[i])
    returning[i]=re.sub(' 00:00:00', '', returning[i])

In [ ]:
headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

In [ ]:
data=[]

for i in range(len(departing)):
  print("*************************************************************************************************")
  url ='https://www.ca.kayak.com/flights/' + cities + '/' + departing[i]+ '/' + returning[i] +'?sort=bestflight_a'
  print(url)
  page=requests.get(url,headers=headers)
  print(page)

  soup = BeautifulSoup(page.content, 'html.parser')
 

  my_table = soup.find("span", class_="price-text")
  print(my_table.text)
    
  try:
      data.append(my_table.get_text())
      data[i]=int(re.sub('[^0-9]','', data[i]))
        
      if data[i]<price_goal:
            
          #Email info
          sender = 'airpricefinder@gmail.com'
          password = 'airPriceFinder@123'
          port = 465
          receive = 'airpricefinder@gmail.com'
           
          #Email content
          msg = MIMEMultipart()
          msg['From'] = sender
          msg['To'] = receive
          msg['Subject'] = 'Great deal on tickets found'

          body = 'Go here to see the great deal:' + re.sub('https://www.', '', page)
          msg.attach(MIMEText(body,'plain'))
          text = msg.as_string()

          #Sending email
          context = ssl.create_default_context()
          with smtplib.SMTP_SSL('smtp.gmail.com', port=port, context=context) as server:
              server.login(sender, password)
              server.sendmail(sender, receive, text)
      else:
          None                 
  except:
      data.append('No flights found')


*************************************************************************************************
https://www.ca.kayak.com/flights/YMQ-OSL/2021-12-24/2021-12-31?sort=bestflight_a
<Response [200]>

C$ 1,804 
*************************************************************************************************
https://www.ca.kayak.com/flights/YMQ-OSL/2021-12-24/2022-01-01?sort=bestflight_a
<Response [200]>

C$ 2,415 
*************************************************************************************************
https://www.ca.kayak.com/flights/YMQ-OSL/2021-12-25/2022-01-01?sort=bestflight_a
<Response [200]>

C$ 2,455 
